In [2]:
import quandl
import pandas as pd
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import json
import datetime as dt
import json
import pandas as pd
import math
from datetime import datetime

In [3]:
sp500 = quandl.get("YALE/SPCOMP", authtoken="ckJvd5G2a2dkK9nun3xS")

In [4]:
u = quandl.get("FRED/UNRATENSA", authtoken="ckJvd5G2a2dkK9nun3xS")

In [5]:
def create_columns_rlchange(df):
    df.reset_index(inplace = True)
    df['lag'] = df.iloc[:,1].shift(1)
    to_drop = df.iloc[:1]
    df.drop(to_drop.index, axis=0, inplace=True)
    df['real_change'] = ((df.iloc[:,1]-df.iloc[:,2]))
    df['monthly_return_per'] = (((df.iloc[:,1]-df.iloc[:,2])/df.iloc[:,2])*100)
    df['month'] = df.iloc[:, 0].dt.month
    df['year'] = df.iloc[: ,0].dt.year
    df['monthyear'] = df['year'].map(str) + '/' + df['month'].map(str)
    return df

In [6]:
sp500 = quandl.get("YALE/SPCOMP", authtoken="ckJvd5G2a2dkK9nun3xS")

In [7]:
def create_columns_prcntchange(df):
    df.reset_index(inplace = True)
    df['lag'] = df.iloc[:,1].shift(1)
    to_drop1 = sp500.iloc[:,2:10]
    df.drop(to_drop1, axis=1, inplace=True)
    to_drop = df.iloc[:1]
    df.drop(to_drop.index, axis=0, inplace=True)
    df['monthly_return_per'] = (((df.iloc[:,1]-df.iloc[:,2])/df.iloc[:,2])*100)
    df['month'] = df.iloc[:, 0].dt.month
    df['year'] = df.iloc[: ,0].dt.year
    df['monthyear'] = df['year'].map(str) + '/' + df['month'].map(str)
    return df

In [1]:
def sp_add_presidents(df):
    presidents = pd.read_csv('us_presidents.csv')
    presidents = pd.DataFrame(presidents)
    date_start_series = list(presidents['start'])
    date_end_series = list(presidents['end'])
    date_end_series[-2] = 'January 20, 2017'
    date_end_series[-1] = 'November 12, 2019'
    
    new_date_end_series = []
    for row in date_end_series:
        date = dt.datetime.strptime(row,'%B %d, %Y').strftime('%Y-%m-%d')
        datetime_object = datetime.strptime(date, '%Y-%m-%d')
        new_date_end_series.append(datetime_object)
        
    presidents['end_date'] = new_date_end_series  
    
    new_president_df = presidents[['president', 'end_date']].copy()
        
    new_president_df['month'] = new_president_df['end_date'].dt.month
    new_president_df['year'] = new_president_df['end_date'].dt.year
    new_president_df['monthyear'] = new_president_df['year'].map(str) + '/' + new_president_df['month'].map(str)
    
    sp500['Value'] = sp500['S&P Composite']
    sp500['Date'] = sp500['Year']
    df_n = df.merge(new_president_df, on='monthyear', how='left')
    df_new = df_n[['Date','Value','diff', 'president']]
    df_new['president'] = df_n['president'].fillna(method='backfill')
    df_new.president.fillna('Donald Trump', inplace = True)
    
    return df_new
    